In [1]:
import pandas as pd 
import numpy as np 

In [2]:
path =""
train_Data = pd.read_csv(path + "source.csv", header=None)


In [3]:
train_Data

,0,1
0,1,嘎嫂新髮型宛如高中生 蔡阿嘎合照根本「誘拐未成年少女」
1,2,美國報告未提證據 俄：操縱大選是無中生有
2,3,新年搶優惠 通路Apple商品加碼送
3,4,台灣癌症時鐘加快 每5分26秒就1人罹癌
4,5,藍營胡批兩岸急凍壓垮興航 綠委回嗆「不用腦子的政黨」
5,6,食藥署提農藥殘留容許量審查原則 學界認為不合理
6,7,肩膀動不了 這療法讓勞動媽歡度母親節
7,8,若再談通俄門「川普可能陷險境」
8,9,【結盟片】搶救歐盟大作戰 「梅克宏」聯盟成形
9,10,僑教中心拒借場地／黃國昌批沈呂巡 醜陋政治操作


In [4]:
train_Data.head(50)

,0,1
0,1,嘎嫂新髮型宛如高中生 蔡阿嘎合照根本「誘拐未成年少女」
1,2,美國報告未提證據 俄：操縱大選是無中生有
2,3,新年搶優惠 通路Apple商品加碼送
3,4,台灣癌症時鐘加快 每5分26秒就1人罹癌
4,5,藍營胡批兩岸急凍壓垮興航 綠委回嗆「不用腦子的政黨」
5,6,食藥署提農藥殘留容許量審查原則 學界認為不合理
6,7,肩膀動不了 這療法讓勞動媽歡度母親節
7,8,若再談通俄門「川普可能陷險境」
8,9,【結盟片】搶救歐盟大作戰 「梅克宏」聯盟成形
9,10,僑教中心拒借場地／黃國昌批沈呂巡 醜陋政治操作


In [5]:
sentence_list = train_Data.iloc[:,1].values


In [6]:
n_gram = 6

## Building Inverted Index 

In [7]:
import time

## Original soultion

In [8]:
ct = time.time()
n_gram_store = [] #  A list used to store every n_gram dictionary 
for i in range(n_gram):
    gram_dict = {} # A dictionary used to contain set with n-gram index 
    i = i+1 
    ngt = time.time()
    print("Now processing %s gram" %(i))
    #process every sentence in query 
    for sentence_index, sentence in enumerate(sentence_list):
        sentence = list(sentence)
        #Start extract n-gram to gram_list:
        for j in range(len(sentence) - (i-1)): # Preventing "index out of range" error
        
            # Concat char to get N-gram index
            gram_index = ''.join(sentence[j:j+i])
            
            if gram_index not in gram_dict:
                gram_dict[gram_index] = set([sentence_index+1]) # index_offset ->plus one 
            else : 
                gram_dict[gram_index].add(sentence_index+1) # index_offset ->plus one
    n_gram_store.append(gram_dict)
    print("For", i, "Cost", time.time() - ngt)
print(time.time() - ct)

Now processing 1 gram
For 1 Cost 1.7609212398529053
Now processing 2 gram
For 2 Cost 3.8771839141845703
Now processing 3 gram
For 3 Cost 5.400454044342041
Now processing 4 gram
For 4 Cost 5.139646053314209
Now processing 5 gram
For 5 Cost 3.478955030441284
Now processing 6 gram
For 6 Cost 5.022411108016968
24.680745840072632


## Much more faster

In [9]:
ct = time.time()
n_gram_store = [] #  A list used to store every n_gram dictionary 
for i in range(n_gram):
    gram_dict = {} # A dictionary used to contain set with n-gram index 
    i = i + 1 
    ngt = time.time()
    print("Now processing %s gram" %(i))
    #process every sentence in query 
    sentence_index = 0
    for sentence in sentence_list:
        #Start extract n-gram to gram_list:
        for j in range(len(sentence) - (i-1)): # Preventing "index out of range" error
            # Concat char to get N-gram index
            key = sentence[j:j+i]
            if key not in gram_dict:
                gram_dict[key] = set([sentence_index+1]) # index_offset ->plus one 
            else : 
                gram_dict[key].add(sentence_index+1) # index_offset ->plus one
        sentence_index += 1
    n_gram_store.append(gram_dict)
    print("For", i, "Cost", time.time() - ngt)
print(time.time() - ct)

Now processing 1 gram
For 1 Cost 1.338047981262207
Now processing 2 gram
For 2 Cost 2.0287749767303467
Now processing 3 gram
For 3 Cost 2.6067042350769043
Now processing 4 gram
For 4 Cost 4.623089075088501
Now processing 5 gram
For 5 Cost 3.890744686126709
Now processing 6 gram
For 6 Cost 3.4053008556365967
18.877669095993042


## Do you think add one is acceptable?

In [10]:
ct = time.time()
n_gram_store = [] #  A list used to store every n_gram dictionary 
for i in range(n_gram):
    gram_dict = {} # A dictionary used to contain set with n-gram index 
    i = i + 1 
    ngt = time.time()
    print("Now processing %s gram" %(i))
    #process every sentence in query 
    sentence_index = 1
    for sentence in sentence_list:
        #Start extract n-gram to gram_list:
        for j in range(len(sentence) - (i-1)): # Preventing "index out of range" error
            # Concat char to get N-gram index
            key = sentence[j:j+i]
            if key not in gram_dict:
                gram_dict[key] = set([sentence_index]) # index_offset ->plus one 
            else : 
                gram_dict[key].add(sentence_index) # index_offset ->plus one
        sentence_index += 1
    n_gram_store.append(gram_dict)
    print("For", i, "Cost", time.time() - ngt)
print(time.time() - ct)

Now processing 1 gram
For 1 Cost 1.364914894104004
Now processing 2 gram
For 2 Cost 2.0260069370269775
Now processing 3 gram
For 3 Cost 4.176756858825684
Now processing 4 gram
For 4 Cost 4.111236095428467
Now processing 5 gram
For 5 Cost 3.9083120822906494
Now processing 6 gram
For 6 Cost 2.9797239303588867
19.712038040161133


## I can boost it without modifying the algorithm

In [11]:
from collections import defaultdict

In [12]:
ct = time.time()
n_gram_store = [] #  A list used to store every n_gram dictionary 
for i in range(n_gram):
    gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
    i = i + 1 
    ngt = time.time()
    print("Now processing %s gram" %(i))
    #process every sentence in query 
    sentence_index = 1
    for sentence in sentence_list:
        #Start extract n-gram to gram_list:
        for j in range(len(sentence) - (i-1)): # Preventing "index out of range" error
            # Concat char to get N-gram index
            key = sentence[j:j+i]
            gram_dict[key].add(sentence_index) # index_offset ->plus one
        sentence_index += 1
    n_gram_store.append(gram_dict)
    print("For", i, "Cost", time.time() - ngt)
print(time.time() - ct)

Now processing 1 gram
For 1 Cost 1.143247127532959
Now processing 2 gram
For 2 Cost 2.2577128410339355
Now processing 3 gram
For 3 Cost 3.5539391040802
Now processing 4 gram
For 4 Cost 3.7982540130615234
Now processing 5 gram
For 5 Cost 3.7093958854675293
Now processing 6 gram
For 6 Cost 3.0811500549316406
18.25630283355713


## a very tricky improvment

In [23]:
lb = 50000
ct = time.time()
n_gram_store = [] #  A list used to store every n_gram dictionary 
for i in range(n_gram):
    gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
    i = i + 1 
    ngt = time.time()
    print("Now processing %s gram" %(i))
    #process every sentence in query 
    sentence_index = 1
    for sentence in sentence_list:
        #Start extract n-gram to gram_list:
        for j in range(len(sentence) - (i-1)): # Preventing "index out of range" error
            # Concat char to get N-gram index
            key = sentence[j:j+i]
            gram_dict[key].add(sentence_index) # index_offset ->plus one
        sentence_index += 1
        if len(gram_dict) > lb:
            print("FULL!")
            n_gram_store.append(gram_dict)
            gram_dict = defaultdict(set)
    print("For", i, "Cost", time.time() - ngt)
print(time.time() - ct)

Now processing 1 gram
For 1 Cost 1.261538028717041
Now processing 2 gram
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
For 2 Cost 1.671464204788208
Now processing 3 gram
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
For 3 Cost 3.0759401321411133
Now processing 4 gram
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
For 4 Cost 3.26853084564209
Now processing 5 gram
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
FULL!
For 5 Cost 3.6791961193084717
Now processing 6 gram
FU

## If you do not like the tricky one, unroll the loop

In [14]:
ct = time.time()
n_gram_store = [] #  A list used to store every n_gram dictionary 


gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
ngt = time.time()
print("Now processing %s gram" %(1))
#process every sentence in query 
sentence_index = 1
for sentence in sentence_list:
    #Start extract n-gram to gram_list:
    for key in sentence:
        gram_dict[key].add(sentence_index) # index_offset ->plus one
    sentence_index += 1
n_gram_store.append(gram_dict)
print("For", 1, "Cost", time.time() - ngt)

gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
ngt = time.time()
print("Now processing %s gram" %(2))
#process every sentence in query 
sentence_index = 1
for sentence in sentence_list:
    #Start extract n-gram to gram_list:
    for j in range(len(sentence) - 1): # Preventing "index out of range" error
        # Concat char to get N-gram index
        key = sentence[j:j+2]
        gram_dict[key].add(sentence_index) # index_offset ->plus one
    sentence_index += 1
n_gram_store.append(gram_dict)
print("For", 2, "Cost", time.time() - ngt)

gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
ngt = time.time()
print("Now processing %s gram" %(3))
#process every sentence in query 
sentence_index = 1
for sentence in sentence_list:
    #Start extract n-gram to gram_list:
    for j in range(len(sentence) - 2): # Preventing "index out of range" error
        # Concat char to get N-gram index
        key = sentence[j:j+3]
        gram_dict[key].add(sentence_index) # index_offset ->plus one
    sentence_index += 1
n_gram_store.append(gram_dict)
print("For", 3, "Cost", time.time() - ngt)

gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
ngt = time.time()
print("Now processing %s gram" %(4))
#process every sentence in query 
sentence_index = 1
for sentence in sentence_list:
    #Start extract n-gram to gram_list:
    for j in range(len(sentence) - 3): # Preventing "index out of range" error
        # Concat char to get N-gram index
        key = sentence[j:j+4]
        gram_dict[key].add(sentence_index) # index_offset ->plus one
    sentence_index += 1
n_gram_store.append(gram_dict)
print("For", 4, "Cost", time.time() - ngt)

gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
ngt = time.time()
print("Now processing %s gram" %(5))
#process every sentence in query 
sentence_index = 1
for sentence in sentence_list:
    #Start extract n-gram to gram_list:
    for j in range(len(sentence) - 4): # Preventing "index out of range" error
        # Concat char to get N-gram index
        key = sentence[j:j+5]
        gram_dict[key].add(sentence_index) # index_offset ->plus one
    sentence_index += 1
n_gram_store.append(gram_dict)
print("For", 5, "Cost", time.time() - ngt)

gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
ngt = time.time()
print("Now processing %s gram" %(6))
#process every sentence in query 
sentence_index = 1
for sentence in sentence_list:
    #Start extract n-gram to gram_list:
    for j in range(len(sentence) - 5): # Preventing "index out of range" error
        # Concat char to get N-gram index
        key = sentence[j:j+6]
        gram_dict[key].add(sentence_index) # index_offset ->plus one
    sentence_index += 1
n_gram_store.append(gram_dict)
print("For", 6, "Cost", time.time() - ngt)


print(time.time() - ct)

Now processing 1 gram
For 1 Cost 0.7895407676696777
Now processing 2 gram
For 2 Cost 1.9973411560058594
Now processing 3 gram
For 3 Cost 3.902238130569458
Now processing 4 gram
For 4 Cost 4.028369903564453
Now processing 5 gram
For 5 Cost 2.537585735321045
Now processing 6 gram
For 6 Cost 2.521742105484009
16.359434127807617


## Why not use multithread ?

In [24]:
import queue, time, threading

In [25]:
exit_flag = -1

In [26]:
class Unit(threading.Thread):
    
    def __init__(self, n):
        threading.Thread.__init__(self)
        self.n = n

    def run(self):
        gram_dict = defaultdict(set) # A dictionary used to contain set with n-gram index 
        i = self.n + 1 
        ngt = time.time()
        print("Now processing %s gram" %(i))
        #process every sentence in query 
        sentence_index = 1
        for sentence in sentence_list:
            #Start extract n-gram to gram_list:
            for j in range(len(sentence) - (i-1)): # Preventing "index out of range" error
                # Concat char to get N-gram index
                key = sentence[j:j+i]
                gram_dict[key].add(sentence_index) # index_offset ->plus one
            sentence_index += 1
            if len(gram_dict) > lb:
                n_gram_store.append(gram_dict)
                gram_dict = defaultdict(set)
        print("For", i, "Cost", time.time() - ngt)

In [27]:
ct = time.time()
units = [Unit(i) for i in range(n_gram)]

for unit in units:
    unit.start()
for t in units:
    t.join()

print("Overall", time.time() - ct)

Now processing 4 gramNow processing 1 gramNow processing 3 gramNow processing 2 gramNow processing 5 gramNow processing 6 gram





For 1 Cost 11.668967008590698
For 6 Cost 11.778158187866211For
 5 Cost 13.538960218429565
For 4 ForCost  217.14626407623291 
CostFor  17.252221822738647Overall3
  17.322908878326416Cost
 17.32063388824463


## Query 

In [19]:
query1 = "美國"
query2 = "北韓"
relation = "and"

first_set = n_gram_store[len(query1)-1][query1]
second_set = n_gram_store[len(query2)-1][query2]

In [20]:
if relation =="and": 
    print(sorted(list(first_set & second_set)))

[44, 5245, 6400, 8233, 9961, 13064, 13488, 14186, 20772, 21518, 24188, 24459, 34511, 38394, 39844, 40906, 41297, 43759, 44807, 45182, 46802, 48123, 52060, 54258, 55426, 57457, 61536, 62168, 64646, 65566, 69953, 70382, 72453, 72837, 76381, 77044, 78502, 80439, 82100, 82776, 88876, 96540, 98546, 98964, 99290]


In [21]:
query1 = "川普"
query2 = "美國"
relation = "and"

first_set = n_gram_store[len(query1)-1][query1]
second_set = n_gram_store[len(query2)-1][query2]


In [22]:
if relation =="and": 
    print(sorted(list(first_set & second_set)))

[13, 678, 1225, 1905, 2756, 5218, 5839, 8690, 9409, 9640, 13064, 13961, 14156, 14186, 14481, 16770, 17606, 18090, 19914, 20105, 22030, 26636, 28769, 29177, 34604, 34995, 35007, 35463, 36249, 39440, 40672, 41297, 42578, 43556, 47121, 48139, 49384, 51361, 51696, 52696, 53362, 55588, 59223, 60384, 60388, 64906, 65735, 65739, 66178, 66507, 72616, 72838, 74469, 76031, 79419, 79806, 80345, 81268, 83306, 86922, 86944, 88801, 91921, 92423, 95643, 98797]
